In [4]:
pip install inflect

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install -U textblob

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install elasticsearch==7.0.0

In [1]:
import time
import inflect
import re
import string
import nltk

from nltk.corpus import stopwords # used for preprocessing
from nltk.stem import WordNetLemmatizer # used for preprocessing
from nltk.stem import PorterStemmer
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from operator import add
from kafka import KafkaConsumer
import json
from textblob import TextBlob

In [2]:
from elasticsearch import Elasticsearch
# create instance of elasticsearch
es = Elasticsearch([{'host':'localhost','port':9200}])

In [3]:
es.ping()

True

In [4]:
topic_name = 'skype'
# ------------------- your consummer ------------------- -----
consumer = KafkaConsumer(
    topic_name,
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='latest',
     enable_auto_commit=True,
     auto_commit_interval_ms =  5000,
     fetch_max_bytes = 128,
     max_poll_records = 100,
     )

In [5]:
#-------------------Traitement du data -----------------------

def remove_urls(text):
    new_text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^A-Za-z# \t])|(\w+:\/\/\S+)"," ",text).split())
    return new_text

# make all text lowercase
def text_lowercase(text):
    return text.lower()

# make all text Uppercase

def text_uppercase(text):
    return text.upper()

In [6]:
#----------delete numbers from text ---------

def remove_numbers(text):
    result = re.sub(r'\d+', '', text)
    return result

In [7]:
# --------------- delete ponctuation existent on text ----------

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


# ----------------convert list to words ---------------
def tokenize(text):
    text = word_tokenize(text)
    return text

In [8]:
#----------- delete words non significant ---------
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    text = [i for i in text if not i in stop_words]
    return text

In [9]:
# ---------------get the origins of words--------------
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    text = [lemmatizer.lemmatize(token) for token in text]
    return text

In [10]:
# -------------remplace all number with word notation-----------
def numbers_to_char(text):
    wordtoken = nltk.word_tokenize(text)
    p = inflect.engine()
    new_Text = []
    for word in wordtoken:
        if word.isdigit():
            newword = p.number_to_words(word)
            new_Text.append(newword)
        else:
            new_Text.append(word)
    return new_Text

In [11]:
#--------------------------end analyse feedback using textBlob--------------------------------
def analyze_sentiment(text):
    testimonial = TextBlob(text)
    return testimonial.sentiment.polarity
def get_sentiment_tuple(sent):
    neutral_threshold = 0.05
    if sent >= neutral_threshold:       # positive
        return (0, 0, 1),"positive"
    elif sent > -neutral_threshold:     # neutral
        return (0, 1, 0),"neutral"
    else:                               # negative
        return (1, 0, 0),"negative"

In [12]:
counter=0 
def topHashtag(text):
    return text.split(" ")
def getNouns(txt):
    blob = TextBlob(txt)
    print(blob.noun_phrases)

In [ ]:
for msg in consumer:
        #print(str(1));
        time.sleep(2)
      
        data= text_lowercase(msg.value)
        
        
        #------hashtages----------------------------------------------
        dict_data = json.loads(data)
        key="text"
        
        if (key in dict_data.keys()):
            print(dict_data["text"])
            hashtags = re.findall("#[a-zA-Z0-9_]{1,200}", dict_data["text"])
            print("hashtags :",hashtags)
       
        #---feedback-----------------------------------
            tweets=TextBlob(dict_data["text"])
            sentiments=analyze_sentiment(dict_data["text"])

        #----------best and bad tweets ------------------
            frequencetweets= text_lowercase(dict_data["text"])
            frequencetweets=remove_urls(frequencetweets)

        #----------- frequency of words---------------
            filtreData= text_lowercase(dict_data["text"])
            filtredData=remove_urls(filtreData)
            filtredData=remove_punctuation(filtredData)
            filtredData=tokenize(filtredData)
            filtredData=remove_stopwords(filtredData)
            FrequencesMots=lemmatize(filtredData)
  
            print(FrequencesMots)
            analysentimens,feedback=get_sentiment_tuple(sentiments)
            print("val :",analysentimens,"  sentiments : ",feedback)
       
        # ------------------- send all result to elastic search --------------------- 
            
            es.index(index="tweet_index" ,
                    doc_type="test_doc",
                    body={
                            "author": dict_data["user"]["screen_name"],
                            "date": dict_data["created_at"],
                            "message": dict_data["text"], 
                            "sentimentsPer":sentiments,
                            "feedback":feedback,
                            "hashtag":hashtags,
                            "FrequencMots":FrequencesMots,
                            "FrequenceTweets":frequencetweets
                    }
                )
           
             
        #-------------------------- optional ---------------------
        #time.sleep(5)
        #print(str(tweets))
        print('\n')